# Settings

In [ ]:
settings = { 
            'data_size': None, # size of data subset after shuffle is performed
            'test_size': 0.25, # fraction of data set to be assigned as test data
            'del': True, # Delete variables that are no longer needed to proceed in computations to save place
            'random_state': { # Set random states so that the results are repeatable
                'shuffle': 42, # sklearn's shuffle method
                'split': 17 # sklearn's train_test_split method
            }
           }

# Load Data: load dataset, shuffle it and take subset

In [ ]:
PATH = "../../data/"

In [ ]:
data_full = pd.read_csv(f'{PATH}prestige.csv')

In [ ]:
data_full.head()

In [ ]:
from sklearn.utils import shuffle
data = shuffle(data_full, random_state=settings['random_state']['shuffle'])

if (settings['del']):
    del data_full

# Data preparation

## Item Category

Split category_name into main_cat, subcat_1 and subcat_2

In [ ]:
# reference: BuryBuryZymon at https://www.kaggle.com/maheshdadhich/i-will-sell-everything-for-free-0-55
def split_cat(text):
    try: return text.split("/")
    except: return ("No Label", "No Label", "No Label")

In [ ]:
data.loc[:,'main_cat'], data.loc[:,'subcat_1'], data.loc[:,'subcat_2'] = \
zip(*data.loc[:,'category_name'].apply(lambda x: split_cat(x)))
# data.head()

## Numerically represent features

### train_id: copy

Create new DataFrame called data_num for numerical representations

In [ ]:
data_num = pd.DataFrame(data.loc[:,'train_id'], columns=['train_id'])

### name: represent name by its length as name_len

In [ ]:
data_num.loc[:,'name_len'] = data['name'].str.len()

### item_condition_id, price, shipping: copy

In [ ]:
data_num[['item_condition_id', 'price', 'shipping']] = data.loc[:,['item_condition_id', 'price', 'shipping']]

### item_description

Represent item_description by its length as item_description_len

In [ ]:
item_description_len = data.loc[:,'item_description'].str.len()
data_num['item_description_len'] = item_description_len

# Replace NaN in data_num.item_description_len column by zeros
data_num['item_description_len'] = data_num['item_description_len'].fillna(0)

# Change data type of this column to uint16 provided the max val is less than 65535
if (data_num.item_description_len.max() < 65535):
    data_num['item_description_len'] = data_num['item_description_len'].astype(np.uint16)

### prestige, prestige_reliability: copy

In [ ]:
data_num.loc[['prestige', 'prestige_reliability']] = data.loc[['prestige', 'prestige_reliability']]

### Define function for making binary columns

In [ ]:
def make_binary_columns(df_str, df_num, column_name):
    """
    Turns a single column named column_name (with various categories) into m binary columns, where m is the number
    of unique values in the original column. For each sample, the value for all new columns is 0 apart the one
    that matches the value of the original column. Names of new binary columns are formed as follows:
    column_name + '_' + str(original column value)
    Inputs:
        two pandas DataFrames: df_str where a single column contains information about given category
                               df_num that will later contain many binary columns
        column_name: name of the column that will be split into several binary columns
    Returns nothing. It appends the collumns directly into df_num to increase performance.
    """
    m = len(df_str)
    if (m != len(df_num)):
        raise Exception("df_str and df_num must have the same size.")
 
    categories = df_str[column_name].unique()

    from tqdm import tqdm_notebook # progress bar

    # Create a new feature for each category and initialize it to 0
    for i in tqdm_notebook(categories, desc='1/2'):
        df_num[column_name + '_' + str(i)] = np.zeros((m, 1), dtype=np.int8)

#     Loop thorugh all rows and assign 1 to the column whose name is the same as category
    for i in tqdm_notebook(df_str.index, desc='2/2'): # loop through all rows
        category = str(df_str.at[i, column_name])
        df_num.at[i, column_name + '_' + category] = 1

### main_cat, subcat_1, subcat_2: for each unique one create new binary feature

In [ ]:
make_binary_columns(data, data_num, 'main_cat')
make_binary_columns(data, data_num, 'subcat_1')
make_binary_columns(data, data_num, 'subcat_2')

In [ ]:
if (settings['del']):
    del data

# Split data and extract X, y and train_id

In [ ]:
from sklearn.model_selection import train_test_split
X_train_unscaled, X_test_unscaled = train_test_split(data_num, test_size = settings['test_size'], random_state=settings['random_state']['split']) # randomly split data
# ! X_train_unscaled and X_test_unscaled STILL CONTAINS PRICE AT THIS MOMENT !

In [ ]:
if (settings['del']):
    del data_num

In [ ]:
# Pop price from X_train and X_test
y_train = X_train_unscaled.pop('price')
y_test = X_test_unscaled.pop('price')

In [ ]:
# Pop id_train from both training and test data set
id_train = X_train_unscaled.pop('train_id')
id_test = X_test_unscaled.pop('train_id')

# Scale data

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

X_train = X_train_unscaled.copy()
X_test = X_test_unscaled.copy()

columns_to_scale = ['name_len', 'item_condition_id', 'item_description_len']

scaler = MaxAbsScaler()
scaler.fit(X_train_unscaled[columns_to_scale]) # Compute the mean adn std of training data to be used for future scaling

X_train[columns_to_scale] = pd.DataFrame(scaler.transform(X_train_unscaled[columns_to_scale]), index=X_train_unscaled.index, columns=columns_to_scale)
if (settings['del']):
    del X_train_unscaled

X_test[columns_to_scale] = pd.DataFrame(scaler.transform(X_test_unscaled[columns_to_scale]), index=X_test_unscaled.index, columns=columns_to_scale)
if (settings['del']):
    del X_test_unscaled

In [ ]:
# X_train_unscaled.info()

In [ ]:
X_train.info()

In [ ]:
X_train.head()

In [ ]:
# X_test_unscaled.info()

In [ ]:
X_test.head()

In [ ]:
# X_train.to_csv('X_train.csv')
# X_train_scaled.to_csv('X_train_scaled.csv')
# X_test.to_csv('X_test.csv')
# X_test_scaled.to_csv('X_test_scaled.csv')

# Check size of variables

In [42]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' By Fred Cirera, after https://stackoverflow.com/a/1094933/1870254'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name,value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name,sizeof_fmt(size)))

                       X_train:   6.1GiB
                        X_test:   2.0GiB
          item_description_len:  62.6MiB
                       y_train:  17.0MiB
                      id_train:  17.0MiB
                        y_test:   5.7MiB
                       id_test:   5.7MiB
                           ___:  28.6KiB
                           _29:  28.6KiB
                             _:  28.6KiB


# Performance measure: RMSE

$$\text{RMSE} \left( \mathbf{Y} , \mathbf{\hat{Y}} \right) = \sqrt{\frac{1}{n} \sum_{i=1}^n \left( y_i - \hat{y_i} \right)^2 } $$

In [43]:
def rmse(y_test, y_pred):
    from sklearn.metrics import mean_squared_error
    return np.sqrt(mean_squared_error(y_test, y_pred))

# Apply linear regression to the full trainin set, compute training and test RMSE and add them to the file created above

In [ ]:
# Fit linear regression
from sklearn.linear_model import LinearRegression
lnr_regr = LinearRegression(n_jobs=-1)
print("Trainig...")
lnr_regr.fit(X_train, y_train)
print("Training done.")
    
# Make predictions and report train and test RMSEs

print("Evaluating performance on the training set...")
pred_train = lnr_regr.predict(X_train)
rmse_train = rmse(y_train, pred_train)
print("Training set RMSE: %.2f" % rmse_train)

print("Evaluating performance on the test test...")
pred_test = lnr_regr.predict(X_test)
rmse_test = rmse(y_test, pred_test)
print("Test set RMSE: %.2f" % rmse_test)

print("Done.")